In [47]:
!pip install pycuda

In [48]:
import numpy as np # linear algebra
import random
import pycuda.driver as cuda
import pycuda.autoinit
from pycuda.compiler import SourceModule
np.random.seed(5)

def time_sum(kernel_str,size,ls=1024,gs=1):
    fastest = None
    for i in range(20):
        a = np.random.rand(size).astype(np.float32)
        res = np.zeros(1).astype(np.float32)
        a_gpu = cuda.mem_alloc(a.nbytes)
        cuda.memcpy_htod(a_gpu, a)
        res_gpu = cuda.mem_alloc(res.nbytes)
        cuda.memcpy_htod(res_gpu, res)
        mod = SourceModule(eval(f'f"""{kernel_str}"""'))
        func = mod.get_function("sum")
        end=cuda.Event()
        start=cuda.Event()
        start.record()
        func(a_gpu,res_gpu, block=(ls,1,1),grid=(gs,1,1))
        cuda.memcpy_dtoh(res, res_gpu)
        end.record()
        end.synchronize()
        t = start.time_till(end)
        if fastest == None or t < fastest:
            fastest = t
        np.testing.assert_allclose(res,[a.sum()],rtol=1e-4)
    print("time:",fastest,"\tsize:\t",size)


    

In [49]:
print("single thread\n")
for i in range(5):
    time_sum("""
          __global__ void sum(const float *a, float *res)
          {{
            float total = 0;
            for(int i = 0; i < {size}; i++) {{
                total += a[i];
            }}
            res[0] = total;
          }}
          """,2048*10**i,1,1)

single thread

time: 0.06531199812889099 	size:	 2048
time: 0.32019200921058655 	size:	 20480
time: 2.8997440338134766 	size:	 204800
time: 49.287296295166016 	size:	 2048000
time: 492.3429870605469 	size:	 20480000


In [50]:
print("\nmy multi reduce (slow) \n")
for i in range(5):
    time_sum("""
          __global__ void sum(const float *a, float *res)
          {{
            int idx = threadIdx.x+blockIdx.x*blockDim.x;
            __shared__ float temp[{ls}];
            temp[idx] = 0;
            for(int i = idx*{int(size/ls)}; i < idx*{int(size/ls)}+{int(size/ls)}; i++) {{
                temp[idx] += a[i];
            }}
            __syncthreads();
            if(idx == 0) {{
                float total = 0;
                for(int i = 0; i < {ls}; i++) {{
                    total += temp[i];
                }}
                res[0] = total;
            }}
          }}
          """,2048*10**i,1024,1)


my multi reduce (slow) 

time: 0.04303999990224838 	size:	 2048
time: 0.05299200117588043 	size:	 20480
time: 0.20707200467586517 	size:	 204800
time: 2.652928113937378 	size:	 2048000
time: 27.54863929748535 	size:	 20480000


In [51]:
print("\nmulti reduce\n") #from https://www.youtube.com/watch?v=D4l1YMsGNlU
for i in range(5):
    time_sum("""
          __global__ void sum(const float *a, float *res)
          {{
            __shared__ float sdata[{ls}];
            int idx = threadIdx.x;
            sdata[idx] = 0.0f;
            for(int i = threadIdx.x; i < {size}; i+={ls}) {{
                sdata[idx] += a[i];
            }}
            for(int s={ls/2}; s>0; s/=2) {{
                __syncthreads();
                if(idx < s) {{
                    sdata[idx] += sdata[idx+s];
                }}
            }}
            if(idx == 0) {{
                res[blockIdx.x] = sdata[0];
            }}
          }}
          """,2048*10**i,1024,1)


multi reduce

time: 0.04291199892759323 	size:	 2048
time: 0.04368000105023384 	size:	 20480
time: 0.05939200147986412 	size:	 204800
time: 0.39792001247406006 	size:	 2048000
time: 3.3966400623321533 	size:	 20480000


In [52]:
print("\nwarp shuffle\n") #from https://www.youtube.com/watch?v=D4l1YMsGNlU
for i in range(5):
    time_sum("""
          __global__ void sum(const float *a, float *res)
          {{
            __shared__ float sdata[32];
            int tid = threadIdx.x;
            float val = 0;
            unsigned mask = 0xFFFFFFFFU;
            int lane = threadIdx.x % warpSize;
            int warpID = threadIdx.x / warpSize;
            for(int i = threadIdx.x + blockDim.x*blockIdx.x; i < {size}; i+= gridDim.x*blockDim.x) {{
                val += a[i];
            }}
            for(int offset = warpSize/2; offset > 0; offset /= 2)
                val+= __shfl_down_sync(mask,val,offset);
            if(lane == 0) sdata[warpID] = val;
            __syncthreads();
            if(warpID == 0) {{
                if(tid < blockDim.x/warpSize) {{
                    val = sdata[lane];
                }} else {{
                    val = 0;
                }}
                for(int offset = warpSize/2; offset > 0; offset /= 2)
                    val += __shfl_down_sync(mask,val,offset);
                if(tid == 0) atomicAdd(res, val);
            }}
          }}
          """,2048*10**i,1024,int(2048*10**i/1024))


warp shuffle

time: 0.0424639992415905 	size:	 2048
time: 0.04224000126123428 	size:	 20480
time: 0.043296001851558685 	size:	 204800
time: 0.11724799871444702 	size:	 2048000
time: 0.7440959811210632 	size:	 20480000
